In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [7]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [8]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [36. 33. 22. 28. 21.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [1 1 0 0 0]
fare                : [512.329  20.525   7.796   7.75    8.433]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'B' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [9]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [19. 48. 11. 28. 28.]
n_siblings_spouses  : [0 0 1 0 0]
parch               : [0 0 2 0 0]
fare                : [ 10.5     7.854 120.      7.829   7.75 ]
class               : [b'Second' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Queenstown']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 58. 35. 28. 23.]
n_siblings_spouses  : [1. 0. 0. 0. 3.]
parch               : [0. 0. 0. 0. 2.]
fare                : [ 14.454 146.521  10.5    13.    263.   ]


In [11]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [12]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [13]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 25.      1.      0.      7.925]
 [ 40.      0.      0.     31.   ]
 [ 41.      0.      0.      7.125]
 [ 26.      0.      0.     56.496]
 [ 58.      0.      1.    153.462]]

[0 1 0 1 1]


In [14]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [35. 37. 39. 28. 25.]
n_siblings_spouses  : [1 1 0 1 0]
parch               : [1 0 0 0 0]
fare                : [20.25 53.1   0.   24.   13.  ]
class               : [b'Third' b'First' b'First' b'Second' b'Second']
deck                : [b'unknown' b'C' b'A' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'n' b'y' b'n' b'y']


In [15]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [16]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [17]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [18]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
class               : [b'Third' b'Second' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'y' b'y' b'n']
numeric             : [[33.     0.     0.     9.5  ]
 [19.     0.     0.    26.   ]
 [28.     0.     0.    15.5  ]
 [49.     0.     0.     0.   ]
 [24.     0.     3.    19.258]]


In [19]:
example_batch, labels_batch = next(iter(packed_train_data))

In [20]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [21]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [22]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [23]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7ff97c706598>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [24]:
example_batch['numeric']

<tf.Tensor: id=472, shape=(5, 4), dtype=float32, numpy=
array([[34.5  ,  0.   ,  0.   ,  6.438],
       [44.   ,  0.   ,  0.   ,  7.925],
       [25.   ,  1.   ,  0.   ,  7.775],
       [24.   ,  0.   ,  2.   , 16.7  ],
       [71.   ,  0.   ,  0.   , 49.504]], dtype=float32)>

In [25]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.389, -0.474, -0.479, -0.512],
       [ 1.148, -0.474, -0.479, -0.485],
       [-0.37 ,  0.395, -0.479, -0.487],
       [-0.45 , -0.474,  2.043, -0.324],
       [ 3.306, -0.474, -0.479,  0.277]], dtype=float32)

In [26]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [27]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [28]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [29]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]


In [30]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [31]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     1.     0.     0.     0.389 -0.474
 -0.479 -0.512  1.     0.   ]


In [32]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [33]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [34]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 3s 21ms/step - loss: 0.5324 - accuracy: 0.7432
Epoch 2/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4250 - accuracy: 0.8278
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4098 - accuracy: 0.8325
Epoch 4/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3943 - accuracy: 0.8341
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3898 - accuracy: 0.8262
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3758 - accuracy: 0.8373
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3667 - accuracy: 0.8437
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3604 - accuracy: 0.8549
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3518 - accuracy: 0.8517
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3556 - accuracy: 0.853

In [35]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 1s 15ms/step - loss: 0.4296 - accuracy: 0.8447


Test Loss 0.42961967961405806, Test Accuracy 0.8446969985961914


In [36]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 95.09%  | Actual outcome:  SURVIVED
Predicted survival: 29.22%  | Actual outcome:  DIED
Predicted survival: 9.12%  | Actual outcome:  DIED
Predicted survival: 6.67%  | Actual outcome:  DIED
Predicted survival: 70.78%  | Actual outcome:  DIED
